# ADS 508 Final Project

By: Christopher Garcia, Claire Phibbs, and Christine Vu

## Diabetes Health Indicators Dataset:
https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset

## Loading in Necessary Libraries 

In [46]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import boto3
import sagemaker
!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

## Ingesting the Data into AWS S3 and SageMaker

In [47]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name = "sagemaker", region_name = region)

In [48]:
# viewing the s3 bucket content to confirm the 3 csv files are there
!aws s3 ls s3://diabetes-ads508/

                           PRE diabetes_5050_binary_indicators/
                           PRE diabetes_binary_indicators/
                           PRE diabetes_indicators/
2023-03-16 21:45:18   11829910 diabetes_012_health_indicators_BRFSS2015.csv
2023-03-16 21:45:19    3307813 diabetes_binary_5050split_health_indicators_BRFSS2015.csv
2023-03-16 21:45:27   11829913 diabetes_binary_health_indicators_BRFSS2015.csv


In [54]:
# set source location of public s3 bucket
s3_public_path = "s3://diabetes-ads508/diabetes_indicators/"
%store s3_public_path

Stored 's3_public_path' (str)


In [55]:
# set distination location of private s3 bucket
s3_private_path = "s3://{}/diabetes-ads508/diabetes_indicators".format(bucket)
print(s3_private_path)
%store s3_private_path

s3://sagemaker-us-east-1-517122314325/diabetes-ads508/diabetes_indicators
Stored 's3_private_path' (str)


In [56]:
!aws s3 cp --recursive $s3_public_path/ $s3_private_path/ --exclude "*" --include "diabetes_012_health_indicators_BRFSS2015.csv"
#!aws s3 cp --recursive $s3_public_path/ $s3_private_path/ --exclude "*" --include "diabetes_binary_5050split_health_indicators_BRFSS2015.csv"
#!aws s3 cp --recursive $s3_public_path/ $s3_private_path/ --exclude "*" --include "diabetes_binary_health_indicators_BRFSS2015.csv"

In [57]:
# list files in the private s3 bucket
print(s3_private_path)

s3://sagemaker-us-east-1-517122314325/diabetes-ads508/diabetes_indicators


In [59]:
# checking files copied to private bucket successfully
!aws s3 ls $s3_private_path/

### Creating Athena Database

In [60]:
inget_create_athena_db_passed = False

In [61]:
# creae database
database_name = "diabetes"

In [62]:
 # Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [63]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [64]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name) 
print(statement)

CREATE DATABASE IF NOT EXISTS diabetes


In [65]:
# verifying database diabetes has been created
pd.read_sql("""SHOW DATABASES""", conn)

,database_name
0,default
1,diabetes
2,diabetes_indicators


### Registering S3 Bucket with Athena 

In [66]:
database_name = "diabetes"
table_name = "diabetes_indicators"

In [67]:
diabetes_indicators_table = """CREATE EXTERNAL TABLE IF NOT EXISTS diabetes.diabetes_indicators(
                Diabetes_012 int,
                HighBP int,
                HighChol int,
                CholCheck int,
                BMI int,
                Smoker int,
                Stroke int, 
                HeartDisease int,
                PhysActivity int,
                Fruits int,
                Veggies int,
                HvyAlcoholConsump int,
                AnyHealthCare int,
                NoDocbcCost int,
                GenHlth int,
                MentHlth int,
                PhysHlth int,
                DiffWalk int,
                Sex int,
                Age int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\\t' LINES TERMINATED BY '\\n' LOCATION 's3://diabetes-ads508/' 
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name, s3_private_path
)

print(diabetes_indicators_table)

CREATE EXTERNAL TABLE IF NOT EXISTS diabetes.diabetes_indicators(
                Diabetes_012 int,
                HighBP int,
                HighChol int,
                CholCheck int,
                BMI int,
                Smoker int,
                Stroke int, 
                HeartDisease int,
                PhysActivity int,
                Fruits int,
                Veggies int,
                HvyAlcoholConsump int,
                AnyHealthCare int,
                NoDocbcCost int,
                GenHlth int,
                MentHlth int,
                PhysHlth int,
                DiffWalk int,
                Sex int,
                Age int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' LOCATION 's3://diabetes-ads508/' 
TBLPROPERTIES ('skip.header.line.count'='1')


In [68]:
pd.read_sql(diabetes_indicators_table, conn)

""


In [69]:
statement = "SHOW TABLES in diabetes".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,tab_name
0,diabetes_indicators


In [70]:
# confirming creation of table
statement = """SELECT * FROM diabetes.diabetes_indicators LIMIT 10""".format(
    database_name, table_name
)

print(statement)

SELECT * FROM diabetes.diabetes_indicators LIMIT 10


In [71]:
df = pd.read_sql(statement, conn)
df

,diabetes_012,highbp,highchol,cholcheck,bmi,smoker,stroke,heartdisease,physactivity,fruits,veggies,hvyalcoholconsump,anyhealthcare,nodocbccost,genhlth,menthlth,physhlth,diffwalk,sex,age
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


## Loading in the Dataset

In [ ]:
diabetes_indicators = pd.read_csv("./diabetes_012_health_indicators_BRFSS2015.csv")
diabetes_binary = pd.read_csv("./diabetes_binary_health_indicators_BRFSS2015.csv")

In [ ]:
diabetes_indicators.head()    # target variable has 3 classes 0-2

In [ ]:
diabetes_binary.head()     # target variable is binary two class 0-1

In [ ]:
diabetes_binary['Diabetes_binary'].value_counts()

In [ ]:
diabetes_indicators['Diabetes_012'].value_counts()

## Exploratory Data Analysis

### Visualizing the Target Variable Diabetes

In [ ]:
values = ['0', '1']
count = diabetes_binary['Diabetes_binary'].value_counts()

plt.bar(values, diabetes_binary['Diabetes_binary'].value_counts())


